In [1]:
!pip install mujoco
!pip install mujoco-mjx
!pip install mediapy

In [2]:
import numpy as np

import jax
import mujoco
from mujoco import mjx
import mediapy as media

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [3]:
XML = r"""
<mujoco model="spider">
  <default>
    <geom type="capsule" size="0.05 0.1" rgba="0.8 0.6 0.4 1"/>
    <joint type="hinge" limited="true" range="-45 45"/>
  </default>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1="1 1 1" rgb2="0.6 0.8 1" width="256" height="256"/>
  </asset>

  <worldbody>
    <light pos="0 0 2" dir="0 0 -1" diffuse="1 1 1"/>

    <!-- Ground plane -->
    <geom type="plane" pos="0 0 0" size="10 10 0.1" rgba="0.9 0.9 0.9 1"/>

    <!-- Central body -->
    <body name="body" pos="0 0 0.3">
      <joint type="free"/>
      <geom type="sphere" size="0.2"/>

      <!-- Legs -->
      <body name="leg1" pos="0.2 0 0">
        <joint name="body_leg1_joint1" axis="0 1 0"/>
        <geom fromto="0 0 0  0.3 0 0"/>
        <body pos="0.3 0 0">
          <joint name="body_leg1_joint2" axis="0 1 0"/>
          <geom fromto="0 0 0  0.3 0 0"/>
        </body>
      </body>

      <body name="leg2" pos="-0.2 0 0">
        <joint name="body_leg2_joint1" axis="0 1 0"/>
        <geom fromto="0 0 0  -0.3 0 0"/>
        <body pos="-0.3 0 0">
          <joint name="body_leg2_joint2" axis="0 1 0"/>
          <geom fromto="0 0 0  -0.3 0 0"/>
        </body>
      </body>

      <body name="leg3" pos="0 0.2 0">
        <joint name="body_leg3_joint1" axis="1 0 0"/>
        <geom fromto="0 0 0  0 0.3 0"/>
        <body pos="0 0.3 0">
          <joint name="body_leg3_joint2" axis="1 0 0"/>
          <geom fromto="0 0 0  0 0.3 0"/>
        </body>
      </body>

      <body name="leg4" pos="0 -0.2 0">
        <joint name="body_leg4_joint1" axis="1 0 0"/>
        <geom fromto="0 0 0  0 -0.3 0"/>
        <body pos="0 -0.3 0">
          <joint name="body_leg4_joint2" axis="1 0 0"/>
          <geom fromto="0 0 0  0 -0.3 0"/>
        </body>
      </body>
    </body>
  </worldbody>

  <!-- Actuators -->
  <actuator>
    <motor joint="body_leg1_joint1" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg1_joint2" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg2_joint1" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg2_joint2" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg3_joint1" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg3_joint2" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg4_joint1" ctrlrange="-1 1" ctrllimited="true"/>
    <motor joint="body_leg4_joint2" ctrlrange="-1 1" ctrllimited="true"/>
  </actuator>
</mujoco>
"""

model = mujoco.MjModel.from_xml_string(XML)
data = mujoco.MjData(model)

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Add random actuations to the motors
np.random.seed(0)
data.ctrl[:] = np.random.uniform(-1, 1, data.ctrl.shape)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.
